## Install Dependancies

In [9]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/UniversalEmb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/UniversalEmb'
/content/drive/My Drive/UniversalEmb


In [2]:
!wget -c https://repo.continuum.io/archive/Anaconda3-2019.03-Linux-x86_64.sh
!chmod +x Anaconda3-2019.03-Linux-x86_64.sh

--2020-04-04 19:08:17--  https://repo.continuum.io/archive/Anaconda3-2019.03-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
!bash ./Anaconda3-2019.03-Linux-x86_64.sh -b -f -p /usr/local

In [0]:
!conda install faiss-cpu -c pytorch

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import faiss
import numpy as np

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import sent_tokenize
import pandas as pd
from tqdm import tqdm

## Pipline

In [18]:
!ls

Anaconda3-2019.03-Linux-x86_64.sh  train_bert_emb3k.npy  TREC_10.label
augm_emb.npy			   train_bert_emb4k.npy  wiki_bert_emb3k.npy
SQuAD_emb.npy			   train_bert_emb.npy	 wiki_bert_emb4k.npy
SQuAD.txt			   train_emb.npy	 wiki_bert_emb.npy
train_5500.label		   train-v2.0.json	 wikitext-103


In [0]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
!unzip wikitext-103-v1.zip

In [0]:
#to do:
#split wiki and use all parts

In [0]:
def read_wiki(path, n_lines = 100000):
  lines = open(path).readlines()[:n_lines]
  sentences = [ ]
  for i in lines:
    sentences.extend(sent_tokenize(i)) 
  print("#  of extracted  sentences: {}".format(len(sentences)))
  return sentences
wiki_sentences = read_wiki('wikitext-103/wiki.train.tokens', 150000)

#  of extracted  sentences: 354303


In [0]:
squad_q = [i[:-1] for i in open('SQuAD.txt', 'r').readlines()]

In [0]:
n_examples = 5000

In [0]:
labels = [i.split(' ', 1)[0].split(':')[1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
texts = [ i.split(' ', 1)[1][:-1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
train  = pd.DataFrame({'texts':texts, 'labels': labels})

In [0]:
def read_emb(path):
  #return np.array([i.split(' ')[:-1] for i in open(path, 'r').readlines()]).astype('float32')
  return np.load(path).astype('float32')

Check specific metrics for each dataset
Multigenre corpus for augmetation (reddit)

In [0]:
def find_nb(train, augment, k = 1, train_bound = 10,  aug_bound = 20000):
  #lists of values and indicies
  res_d, res_i = [], []
  #batching of augmented corpus
  for aug in tqdm(range(0, augment.shape[0], aug_bound)):
      d, i = [], []
      index = faiss.IndexFlatL2(train.shape[1])   # build the index
      index.add(augment[aug:aug+aug_bound])
      #batching of train samples
      for tr in range(0, train.shape[0], train_bound):
        D, I = index.search(train[tr:tr+train_bound], k)
        d.append(D)
        i.append(I)
      #reshaping of train for current augmented corpus 
      d = np.concatenate(tuple(d))
      i = np.concatenate(tuple(i))
      res_d.append(d)
      res_i.append(i)
  #reshaping of result for all train examples and augmented corpus
  res_d = np.concatenate(tuple(res_d), axis = 1)
  res_i = np.concatenate(tuple(res_i), axis = 1)
  #take needed nbs from result
  return np.take(res_i, np.argsort(res_d, axis = -1)[:, :k]), np.take(res_d, np.argsort(res_d, axis = -1)[:, :k])

In [151]:
augmented_emb = read_emb('SQuAD_emb.npy')
train_emb = read_emb('train_emb.npy')
train_emb=train_emb[:3000]
print(train_emb.shape)
ind, val = find_nb(train_emb, augmented_emb, k=3)

  0%|          | 0/7 [00:00<?, ?it/s]

(3000, 512)


100%|██████████| 7/7 [01:00<00:00,  8.63s/it]


In [152]:
val.shape

(3000, 3)

In [153]:
for i,j  in zip(train[:10].texts.tolist(), train[:10].labels.tolist()):
  print(i, '######', j)

How did serfdom develop in and then leave Russia ? ###### manner
What films featured the character Popeye Doyle ? ###### cremat
How can I find a list of celebrities ' real names ? ###### manner
What fowl grabs the spotlight after the Chinese Year of the Monkey ? ###### animal
What is the full form of .com ? ###### exp
What contemptible scoundrel stole the cork from my lunch ? ###### ind
What team did baseball 's St. Louis Browns become ? ###### gr
What is the oldest profession ? ###### title
What are liver enzymes ? ###### def
Name the scar-faced bounty hunter of The Old West . ###### ind


In [154]:
for i in ind[:10]:
  print('##########')
  for j in i:
    print(squad_q[j])

##########
When were the actors in Spectre announced?
What city is Stewart International Airport located close to?
What was the name of Estêvão Gomes's ship?
##########
 Which countries didn't take part in the Northern Crusades?
What kind of scores did Twilight Princess receive from many video game review sources?
Which Luna probe successfully photographed the back side of the Moon?
##########
Which branches of the national military does the U.S. president command?
 Which countries didn't take part in the Northern Crusades?
When were the actors in Spectre announced?
##########
Which branches of the national military does the U.S. president command?
How did Mikhail Gorbachev begin the era of stagnation?
 Which countries didn't take part in the Northern Crusades?
##########
How did Mikhail Gorbachev begin the era of stagnation?
Anglican church law contributes to what for each of its members?
What city is Stewart International Airport located close to?
##########
 Which countries didn't t

In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

# Model and evaluation

In [193]:
k=0
aug_sentences = []
for augmts_per_class in ind:
  aug_sentences.extend([squad_q[i] for indx, i in enumerate(augmts_per_class.tolist()) if indx<=k-1 ])

aug_labels = []
for i in labels:
  aug_labels+=[i]*k
print(len(aug_sentences))
print(len(aug_labels))
assert len(aug_sentences)==len(aug_labels)

0
0


In [194]:
import pandas as pd
n_examples = 3000

labels = [i.split(' ', 1 )[0].split(':')[1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
texts = [ i.split(' ', 1 )[1][:-1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_examples]
train  = pd.DataFrame({'texts':texts+aug_sentences, 'labels': labels+aug_labels})

labels_t = [i.split(' ', 1)[0].split(':')[1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
test  = pd.DataFrame({'texts':texts_t, 'labels': labels_t})
test.shape

(500, 2)

In [0]:
tr = {i:j for j, i in enumerate(train.labels.unique().tolist())}
def t(f):
  return tr[f]
train['lb'] = train.labels.apply(t)
test['lb'] = test.labels.apply(t)                  

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

In [197]:
tf = TfidfVectorizer(lowercase = False, ngram_range = (1,3), max_df =0.98, min_df = 3)
tf_res = tf.fit_transform(train.texts)
test_transform = tf.transform(test.texts)
tf_res.shape

(3000, 2333)

In [0]:
import numpy as np
from sklearn.metrics import classification_report
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import time

In [0]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

class WordData(Dataset):
    
    def __init__(self, x_data, y_data):
        
        super().__init__()
        
        self.x_data = x_data
        self.y_data = torch.LongTensor([int(i) for i in y_data])
        
    
    def __len__(self):
        
        return self.x_data.shape[0]
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx].toarray()
        x = torch.FloatTensor(x)
        y = self.y_data[idx]
        
        return x, y


dataset_train = WordData(tf_res, train.lb)
dataset_test = WordData(test_transform, test.lb)

batch_size = 256

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,  shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [0]:
from math import sqrt
class MLP(torch.nn.Module):
    
    def __init__(self, in_features, inner_features, inner_features2, out_features):
        
        # Главная задача этого метода - возможность использования в классе потомке, методов класса-родителя
        # Вызываем __init__ родителя - torch.nn.Module
        super().__init__()
        
        # определяем слои и все что захотим сохранять/использовать
        self.linear_1 = torch.nn.Linear(in_features=in_features, out_features=inner_features)
        gl = sqrt(6./(in_features+inner_features))
        self.linear_1.weight.data.uniform_(-gl, gl)
        self.non_linear_function = torch.nn.ReLU()
        self.drop_out = torch.nn.Dropout(0.18)
        self.linear_2 = torch.nn.Linear(in_features=inner_features, out_features=inner_features2)
        gl1 = sqrt(6./(inner_features+inner_features2))
        self.linear_2.weight.data.uniform_(-gl1, gl1)
        self.non_linear_function2 = torch.nn.ReLU()
        self.linear_3 = torch.nn.Linear(in_features=inner_features2, out_features=out_features)
        gl2 = sqrt(6./(inner_features2+out_features))
        self.linear_3.weight.data.uniform_(-gl2, gl2)
  
    def forward(self, x):
        
        # наша привычная схема
        x = self.linear_1(x)
        x = self.non_linear_function(x)
        x = self.drop_out(x)
        x = self.linear_2(x)
        x = self.non_linear_function2(x)
        x = self.linear_3(x)
        
        return x
    

In [0]:
model = MLP(in_features=tf_res.shape[1], inner_features=524, inner_features2=256, out_features=len(set(train.lb)))

In [0]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0005,  betas=(0.9, 0.999) )
# aka loss function
criterion = torch.nn.CrossEntropyLoss()

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [204]:
from sklearn.metrics import f1_score
epochs = 35
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        pred = model(x).squeeze(1)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:     
        
        x = x.to(device)
        y = y.to(device)
       

        with torch.no_grad():
            pred = model(x).squeeze(1)

            pred = pred.cpu()
            y = y.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.7f}, test - {:.7f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.4f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 2:  17%|█▋        | 512/3000 [00:00<00:00, 3126.86it/s, train_loss=3.76]


Losses: train - 3.7806854, test - 3.6661844
F1 test - 0.3600


Epoch 3:  17%|█▋        | 512/3000 [00:00<00:00, 3107.51it/s, train_loss=3.64]


Losses: train - 3.5505307, test - 3.3245149
F1 test - 0.3800


Epoch 4:   9%|▊         | 256/3000 [00:00<00:01, 2388.41it/s, train_loss=3.44]

...


Losses: train - 0.2237161, test - 1.1006332
F1 test - 0.7580


Epoch 25:  17%|█▋        | 512/3000 [00:00<00:00, 3201.50it/s, train_loss=1.26]


Losses: train - 0.2032400, test - 1.0969205
F1 test - 0.7600


Epoch 25: 100%|██████████| 3000/3000 [00:00<00:00, 3081.05it/s, train_loss=1.23]



Losses: train - 0.1835082, test - 1.1033335
F1 test - 0.7660
Early stopping


In [147]:
model

MLP(
  (linear_1): Linear(in_features=3257, out_features=524, bias=True)
  (non_linear_function): ReLU()
  (drop_out): Dropout(p=0.18, inplace=False)
  (linear_2): Linear(in_features=524, out_features=256, bias=True)
  (non_linear_function2): ReLU()
  (linear_3): Linear(in_features=256, out_features=47, bias=True)
)

In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

In [0]:
#to do
#построцессинг агментации - чтоб не к каждому предложению агментация а по какому нибудь барьеру расстояния - например выше среднего значения - повышение качества агументации, уменьшения данных для тренировки 